In [ ]:
!rm -rf sample_data/  
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/740_deeplearning/* /content/

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


In [ ]:
import os
import warnings
import torch
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack import AutoAttack

# Load model
def load_model(model_path):
    model = ResNet18()
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt)
    model.cuda()
    model.eval()
    return model

# Load data
def load_data(data_dir):
    transform_chain = transforms.Compose([transforms.ToTensor()])
    item = datasets.CIFAR10(root=data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)
    return test_loader

# Create save directory
def create_save_dir(save_dir):
    os.makedirs(save_dir, exist_ok=True)

# Get attack
def get_attack(model, norm, epsilon, log_path, version):
    adversary = AutoAttack(model, norm=norm, eps=epsilon,
                           log_path=log_path, version=version)
    return adversary

def main():
    warnings.filterwarnings("ignore")

    data_dir = './data'
    norm = 'L2'  # Linf
    epsilon = 8. / 255.
    model_path = './model-cifar-ResNet18/model-ResNet18-cifar10-best_epoch.pt'
    individual = False
    save_dir = f'./standard/AA_results_cifar10_{norm}'
    batch_size = 500
    version = 'standard'
    state_path = None

    create_save_dir(save_dir)
    log_path = os.path.join(save_dir, 'log_file.txt')

    model = load_model(model_path)
    test_loader = load_data(data_dir)
    n_ex = len(test_loader.dataset)

    # Get data and labels from test_loader
    x_test = torch.cat([x for (x, y) in test_loader], 0)
    y_test = torch.cat([y for (x, y) in test_loader], 0)
    
    adversary = get_attack(model, norm, epsilon, log_path, version)

    with torch.no_grad():
        if not individual:

            adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex],
                                                              bs=batch_size, state_path=state_path)

            torch.save({'adv_complete': adv_complete}, f'{save_dir}/aa_{version}_1_{adv_complete.shape[0]}_eps_{epsilon:.5f}.pth')
            
        else:

            adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex],
                                                                        y_test[:n_ex], bs=batch_size)
            
            torch.save(adv_complete, f'{save_dir}/aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}.pth')

if __name__ == '__main__':
    main()

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Load the original CIFAR-10 dataset
data_dir = './data'
transform_chain = transforms.Compose([transforms.ToTensor()])
cifar10 = datasets.CIFAR10(root=data_dir, train=False, transform=transform_chain, download=True)

# Define a variable for the number of images to display
num_images = 5

# Select random images from the dataset
random_indices = random.sample(range(len(cifar10)), num_images)
orig_imgs = [cifar10[i][0] for i in random_indices]

# Load the adversarial images as a PyTorch tensor
filename = './standard/AA_results_cifar10_L2/aa_standard_1_10000_eps_0.03137.pth'
data = torch.load(filename)
adv_imgs = [data['adv_complete'][i] for i in random_indices]

# Function to display images using matplotlib
def display_images(imgs, title, row=1):
    for i, img in enumerate(imgs):
        img = img.permute(1, 2, 0)  # Transpose the image from (C, H, W) to (H, W, C)
        img = (img.cpu().numpy() * 255).astype(int)

        plt.subplot(2, num_images, row * num_images + i + 1)
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')

# Display original images and adversarial images side by side
plt.figure(figsize=(3 * num_images, 6))
display_images(orig_imgs, 'Original', row=0)
display_images(adv_imgs, 'Adversarial', row=1)
plt.savefig('./standard/AA_results_cifar10_L2/comparison.png', dpi=300, bbox_inches='tight')
plt.show()